In [1]:
##4QAM, 16QAM, 64QAM, BPSK, 8PSK.

In [371]:
import keras
from keras import backend as K
from keras.layers import Layer
from keras.layers import Activation, Dense
from keras.models import Sequential
from keras import optimizers, regularizers
import numpy as np
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt

In [340]:
#Definitions to automatically generate symbols of QAM and M-Ary PSK
def gray_code(n):
    if n < 1:
        g = []
    else:
        g = ['0', '1']
        n -= 1
        while n > 0:
            k = len(g)
            for i in range(k-1, -1, -1):
                char = '1' + g[i]
                g.append(char)
            for i in range(k-1, -1, -1):
                g[i] = '0' + g[i]
            n -= 1
    return g

class BasicModem:
    def TX(self, payload_data): #String type input
        syms_out = np.zeros(int(len(payload_data)/self.bit_num), dtype=np.complex64)
        cnt = 0
        for i in range(0, len(payload_data) - self.bit_num, self.bit_num):
            syms_out[cnt] = self.inv_dict[payload_data[i:i+self.bit_num]]
            cnt += 1
        return syms_out

class QAM(BasicModem):
    def __init__(self, NUM_SYMBOLS = 4, SAMP_RATE = 1):
        self.NUM_SYMBOLS = NUM_SYMBOLS
        self.SAMP_RATE = SAMP_RATE
        dim_iq_axis = int(np.sqrt(NUM_SYMBOLS))
        self.coords = []
        self.symbols = gray_code(np.log2(NUM_SYMBOLS))
        MAX_ENERGY = np.sqrt(2) #For QAM with coordinates at (x=1,y=1), normalized energy: divide by sqrt(2)
        x = np.linspace(-1, 1, dim_iq_axis, dtype='complex64')
        x, y = np.meshgrid(x, x)
        self.coords = (x + y * 1j).flatten()
        self.dict = dict(zip(self.coords, self.symbols))
        self.inv_dict = dict(zip(self.symbols, self.coords))
        self.bit_num = int(np.log2(NUM_SYMBOLS))

class PSK(BasicModem):
    def __init__(self, NUM_SYMBOLS = 8, SAMP_RATE = 1):
        self.NUM_SYMBOLS = NUM_SYMBOLS
        self.SAMP_RATE = SAMP_RATE
        self.coords = np.zeros(NUM_SYMBOLS, dtype='complex64')
        self.symbols = gray_code(np.log2(NUM_SYMBOLS))
        cnt = 0
        for i in np.arange(0, 2*np.pi, 2*np.pi/NUM_SYMBOLS):
            #i is the phase
            self.coords[cnt] =  1*np.exp(1j*i)
            cnt += 1
        #Above generates an array with the approximate locations of the symbols.
        self.dict = dict(zip(self.coords, self.symbols))
        self.inv_dict = dict(zip(self.symbols, self.coords))
        self.bit_num = int(np.log2(NUM_SYMBOLS))
        print(self.dict)
        


In [341]:
myqam4 = QAM(NUM_SYMBOLS=4)
myqam16 = QAM(NUM_SYMBOLS=16)
myqam64 = QAM(NUM_SYMBOLS=64)
myBPSK = PSK(NUM_SYMBOLS = 2)
myPSK8 = PSK(NUM_SYMBOLS = 16)
myqam4.TX('100100')

{(1+0j): '0', (-1+1.2246469e-16j): '1'}
{(1+0j): '0000', (0.9238795+0.38268343j): '0001', (0.70710677+0.70710677j): '0011', (0.38268343+0.9238795j): '0010', (6.123234e-17+1j): '0110', (-0.38268343+0.9238795j): '0111', (-0.70710677+0.70710677j): '0101', (-0.9238795+0.38268343j): '0100', (-1+1.2246469e-16j): '1100', (-0.9238795-0.38268343j): '1101', (-0.70710677-0.70710677j): '1111', (-0.38268343-0.9238795j): '1110', (-1.8369701e-16-1j): '1010', (0.38268343-0.9238795j): '1011', (0.70710677-0.70710677j): '1001', (0.9238795-0.38268343j): '1000'}


array([1.+1.j, 1.-1.j, 0.+0.j], dtype=complex64)

In [342]:
# Authors: CommPy contributors
# License: BSD 3-Clause

"""
==================================================
Modulation Demodulation (:mod:`commpy.modulation`)
==================================================
.. autosummary::
   :toctree: generated/
   PSKModem             -- Phase Shift Keying (PSK) Modem.
   QAMModem             -- Quadrature Amplitude Modulation (QAM) Modem.
   ofdm_tx              -- OFDM Transmit Signal Generation
   ofdm_rx              -- OFDM Receive Signal Processing
   mimo_ml              -- MIMO Maximum Likelihood (ML) Detection.
   kbest                -- MIMO K-best Schnorr-Euchner Detection.
   bit_lvl_repr         -- Bit level representation.
   max_log_approx       -- Max-log approximation.
"""
from itertools import product

import matplotlib.pyplot as plt
from numpy import arange, array, zeros, pi, cos, sin, sqrt, log2, argmin, \
    hstack, repeat, tile, dot, shape, concatenate, exp, \
    log, vectorize, empty, eye, kron, inf
from numpy.fft import fft, ifft
from numpy.linalg import qr, norm

from commpy.utilities import bitarray2dec, dec2bitarray

__all__ = ['PSKModem', 'QAMModem', 'ofdm_tx', 'ofdm_rx', 'mimo_ml', 'kbest', 'bit_lvl_repr', 'max_log_approx']


class Modem:
    def modulate(self, input_bits):
        """ Modulate (map) an array of bits to constellation symbols.
        Parameters
        ----------
        input_bits : 1D ndarray of ints
            Inputs bits to be modulated (mapped).
        Returns
        -------
        baseband_symbols : 1D ndarray of complex floats
            Modulated complex symbols.
        """
        mapfunc = vectorize(lambda i:
                            self.constellation[bitarray2dec(input_bits[i:i + self.num_bits_symbol])])

        baseband_symbols = mapfunc(arange(0, len(input_bits), self.num_bits_symbol))

        return baseband_symbols

    def demodulate(self, input_symbols, demod_type, noise_var=0):
        """ Demodulate (map) a set of constellation symbols to corresponding bits.
        Parameters
        ----------
        input_symbols : 1D ndarray of complex floats
            Input symbols to be demodulated.
        demod_type : string
            'hard' for hard decision output (bits)
            'soft' for soft decision output (LLRs)
        noise_var : float
            AWGN variance. Needs to be specified only if demod_type is 'soft'
        Returns
        -------
        demod_bits : 1D ndarray of ints
            Corresponding demodulated bits.
        """
        if demod_type == 'hard':
            index_list = map(lambda i: argmin(abs(input_symbols[i] - self.constellation)),
                             range(0, len(input_symbols)))
            demod_bits = array([dec2bitarray(i, self.num_bits_symbol) for i in index_list]).reshape(-1)

        elif demod_type == 'soft':
            demod_bits = zeros(len(input_symbols) * self.num_bits_symbol)
            for i in arange(len(input_symbols)):
                current_symbol = input_symbols[i]
                for bit_index in arange(self.num_bits_symbol):
                    llr_num = 0
                    llr_den = 0
                    for const_index in self.symbol_mapping:
                        if (const_index >> bit_index) & 1:
                            llr_num = llr_num + exp(
                                (-abs(current_symbol - self.constellation[const_index]) ** 2) / noise_var)
                        else:
                            llr_den = llr_den + exp(
                                (-abs(current_symbol - self.constellation[const_index]) ** 2) / noise_var)
                    demod_bits[i * self.num_bits_symbol + self.num_bits_symbol - 1 - bit_index] = log(llr_num / llr_den)
        else:
            raise ValueError('demod_type must be "hard" or "soft"')

        return demod_bits

    def plot_constellation(self):
        """ Plot the constellation """
        # init some arrays
        beta = self.num_bits_symbol
        numbit = '0' + str(beta) + 'b'
        Bin = []
        mot = []
        const = []

        # creation of w array
        reel = [pow(2, i) for i in range(beta // 2 - 1, -1, -1)]
        im = [1j * pow(2, i) for i in range(beta // 2 - 1, -1, -1)]
        w = concatenate((reel, im), axis=None)

        listBin = [format(i, numbit) for i in range(2 ** beta)]
        for e in listBin:
            for i in range(beta):
                Bin.append(ord(e[i]) - 48)
                if ord(e[i]) - 48 == 0:
                    mot.append(-1)
                else:
                    mot.append(1)
            const.append(dot(w, mot))
            mot = []
        symb = self.modulate(Bin)

        # plot the symbols
        x = symb.real
        y = symb.imag

        plt.plot(x, y, '+', linewidth=4)
        for i in range(len(x)):
            plt.text(x[i], y[i], listBin[i])

        plt.title('Constellation')
        plt.grid()
        plt.show()


class PSKModem(Modem):
    """ Creates a Phase Shift Keying (PSK) Modem object. """

    Es = 1

    def _constellation_symbol(self, i):
        return cos(2 * pi * (i - 1) / self.m) + sin(2 * pi * (i - 1) / self.m) * (0 + 1j)

    def __init__(self, m):
        """ Creates a Phase Shift Keying (PSK) Modem object.
        Parameters
        ----------
        m : int
            Size of the PSK constellation.
        """
        self.m = m
        self.num_bits_symbol = int(log2(self.m))
        self.symbol_mapping = arange(self.m)
        self.constellation = list(map(self._constellation_symbol,
                                      self.symbol_mapping))


class QAMModem(Modem):
    """ Creates a Quadrature Amplitude Modulation (QAM) Modem object."""

    def _constellation_symbol(self, i):
        return (2 * i[0] - 1) + (2 * i[1] - 1) * (1j)

    def __init__(self, m):
        """ Creates a Quadrature Amplitude Modulation (QAM) Modem object.
        Parameters
        ----------
        m : int
            Size of the QAM constellation.
        """

        self.m = m
        self.num_bits_symbol = int(log2(self.m))
        self.symbol_mapping = arange(self.m)
        mapping_array = arange(1, sqrt(self.m) + 1) - (sqrt(self.m) / 2)
        self.constellation = list(map(self._constellation_symbol,
                                      list(product(mapping_array, repeat=2))))
        self.Es = 2 * (self.m - 1) / 3


def ofdm_tx(x, nfft, nsc, cp_length):
    """ OFDM Transmit signal generation """

    nfft = float(nfft)
    nsc = float(nsc)
    cp_length = float(cp_length)
    ofdm_tx_signal = array([])

    for i in range(0, shape(x)[1]):
        symbols = x[:, i]
        ofdm_sym_freq = zeros(nfft, dtype=complex)
        ofdm_sym_freq[1:(nsc / 2) + 1] = symbols[nsc / 2:]
        ofdm_sym_freq[-(nsc / 2):] = symbols[0:nsc / 2]
        ofdm_sym_time = ifft(ofdm_sym_freq)
        cp = ofdm_sym_time[-cp_length:]
        ofdm_tx_signal = concatenate((ofdm_tx_signal, cp, ofdm_sym_time))

    return ofdm_tx_signal


def ofdm_rx(y, nfft, nsc, cp_length):
    """ OFDM Receive Signal Processing """

    num_ofdm_symbols = int(len(y) / (nfft + cp_length))
    x_hat = zeros([nsc, num_ofdm_symbols], dtype=complex)

    for i in range(0, num_ofdm_symbols):
        ofdm_symbol = y[i * nfft + (i + 1) * cp_length:(i + 1) * (nfft + cp_length)]
        symbols_freq = fft(ofdm_symbol)
        x_hat[:, i] = concatenate((symbols_freq[-nsc / 2:], symbols_freq[1:(nsc / 2) + 1]))

    return x_hat


def mimo_ml(y, h, constellation):
    """ MIMO ML Detection.
    parameters
    ----------
    y : 1D ndarray of complex floats
        Received complex symbols (shape: num_receive_antennas x 1)
    h : 2D ndarray of complex floats
        Channel Matrix (shape: num_receive_antennas x num_transmit_antennas)
    constellation : 1D ndarray of complex floats
        Constellation used to modulate the symbols
    """
    _, n = h.shape
    m = len(constellation)
    x_ideal = empty((n, pow(m, n)), complex)
    for i in range(0, n):
        x_ideal[i] = repeat(tile(constellation, pow(m, i)), pow(m, n - i - 1))
    min_idx = argmin(norm(y[:, None] - dot(h, x_ideal), axis=0))
    x_r = x_ideal[:, min_idx]

    return x_r


def kbest(y, h, constellation, K, noise_var=0, output_type='hard', demode=None):
    """ MIMO K-best Schnorr-Euchner Detection.
    Reference: Zhan Guo and P. Nilsson, 'Algorithm and implementation of the K-best sphere decoding for MIMO detection',
        IEEE Journal on Selected Areas in Communications, vol. 24, no. 3, pp. 491-503, Mar. 2006.
    Parameters
    ----------
    y : 1D ndarray
        Received complex symbols (length: num_receive_antennas)
    h : 2D ndarray
        Channel Matrix (shape: num_receive_antennas x num_transmit_antennas)
    constellation : 1D ndarray of floats
        Constellation used to modulate the symbols
    K : positive integer
        Number of candidates kept at each step
    noise_var : positive float
        Noise variance.
        *Default* value is 0.
    output_type : str
        'hard': hard output i.e. output is a binary word
        'soft': soft output i.e. output is a vector of Log-Likelihood Ratios.
        *Default* value is 'hard'
    demode : function with prototype binary_word = demode(point)
        Function that provide the binary word corresponding to a symbol vector.
    Returns
    -------
    x : 1D ndarray of constellation points or of Log-Likelihood Ratios.
        Detected vector (length: num_receive_antennas).
    raises
    ------
    ValueError
                If h has more columns than rows.
                If output_type is something else than 'hard' or 'soft'
    """
    nb_tx, nb_rx = h.shape
    if nb_rx > nb_tx:
        raise ValueError('h has more columns than rows')

    # QR decomposition
    q, r = qr(h)
    yt = q.conj().T.dot(y)

    # Initialization
    m = len(constellation)
    nb_can = 1

    if isinstance(constellation[0], complex):
        const_type = complex
    else:
        const_type = float
    X = empty((nb_rx, K * m), dtype=const_type)  # Set of current candidates
    d = tile(yt[:, None], (1, K * m))  # Corresponding distance vector
    d_tot = zeros(K * m, dtype=float)  # Corresponding total distance
    hyp = empty(K * m, dtype=const_type)  # Hypothesis vector

    # Processing
    for coor in range(nb_rx - 1, -1, -1):
        nb_hyp = nb_can * m

        # Copy best candidates m times
        X[:, :nb_hyp] = tile(X[:, :nb_can], (1, m))
        d[:, :nb_hyp] = tile(d[:, :nb_can], (1, m))
        d_tot[:nb_hyp] = tile(d_tot[:nb_can], (1, m))

        # Make hypothesis
        hyp[:nb_hyp] = repeat(constellation, nb_can)
        X[coor, :nb_hyp] = hyp[:nb_hyp]
        d[coor, :nb_hyp] -= r[coor, coor] * hyp[:nb_hyp]
        d_tot[:nb_hyp] += abs(d[coor, :nb_hyp]) ** 2

        # Select best candidates
        argsort = d_tot[:nb_hyp].argsort()
        nb_can = min(nb_hyp, K)  # Update number of candidate

        # Update accordingly
        X[:, :nb_can] = X[:, argsort[:nb_can]]
        d[:, :nb_can] = d[:, argsort[:nb_can]]
        d[:coor, :nb_can] -= r[:coor, coor, None] * hyp[argsort[:nb_can]]
        d_tot[:nb_can] = d_tot[argsort[:nb_can]]

    if output_type == 'hard':
        return X[:, 0]
    elif output_type == 'soft':
        return max_log_approx(y, h, noise_var, X[:, :nb_can], demode)
    else:
        raise ValueError('output_type must be "hard" or "soft"')


def bit_lvl_repr(H, w):
    """ Bit-level representation of matrix H with weights w.
    parameters
    ----------
    H   :   2D ndarray (shape : nb_rx, nb_tx)
            Channel Matrix.
    w   :   1D ndarray of complex (length : beta)
            Bit level representation weights. The length must be even.
    return
    ------
    A : 2D nbarray (shape : nb_rx, nb_tx*beta)
        Channel matrix adapted to the bit-level representation.
    """
    beta = len(w)
    if beta % 2 == 0:
        m, n = H.shape
        In = eye(n, n)
        kr = kron(In, w)
        return dot(H, kr)
    else:
        raise ValueError('Beta must be even.')


def max_log_approx(y, h, noise_var, pts_list, demode):
    """ Max-log demode
    parameters
    ----------
    y : 1D ndarray
        Received symbol vector (length: num_receive_antennas)
    h : 2D ndarray
        Channel Matrix (shape: num_receive_antennas x num_transmit_antennas)
    noise_var : positive float
        Noise variance
    pts_list : 2D ndarray of constellation points
        Set of points to compute max log approximation (points are column-wise).
        (shape: num_receive_antennas x num_points)
    demode : function with prototype binary_word = demode(point)
        Function that provide the binary word corresponding to a symbol vector.
    return
    ------
    LLR : 1D ndarray of floats
        Log-Likelihood Ratio for each bit (same length as the return of decode)
    """
    # Decode all pts
    nb_pts = pts_list.shape[1]
    bits = demode(pts_list.reshape(-1, order='F')).reshape(nb_pts, -1)  # Set of binary words (one word by row)

    # Prepare LLR computation
    nb_bits = bits.shape[1]
    LLR = empty(nb_bits)

    # Loop for each bit
    for k in range(nb_bits):
        # Select pts based on the k-th bit in the corresponding word
        pts0 = pts_list.compress(bits[:, k] == 0, axis=1)
        pts1 = pts_list.compress(bits[:, k] == 1, axis=1)

        # Compute the norms and add inf to handle empty set of points
        norms0 = hstack((norm(y[:, None] - h.dot(pts0), axis=0) ** 2, inf))
        norms1 = hstack((norm(y[:, None] - h.dot(pts1), axis=0) ** 2, inf))

        # Compute LLR
        LLR[k] = min(norms0) - min(norms1)

    return -LLR / (2 * noise_var)

In [343]:
qam4 = QAMModem(4)
qam16 = QAMModem(16)
qam64 = QAMModem(64)
bpsk = PSKModem(2)
psk8 = PSKModem(8)
'''Note that the QAM soft-demodulators simply dont work due to a divide by zero error!'''

'Note that the QAM soft-demodulators simply dont work due to a divide by zero error!'

In [344]:
from scipy import special
def P_error_mqam(M, Es_No_Ratio, Es = None, No = None):
    if Es != None and No != None:
        return 2*(1-1/(np.sqrt(M)))*special.erfc(k*np.sqrt(Es/No))
    else:
        return 2*(1-1/(np.sqrt(M)))*special.erfc(k*np.sqrt(Es_No_Ratio))

def P_error_mpsk(M, Es_No_Ratio, Es=None, No = None):
    if Es != None and No != None:
        return 2 * special.erfc(np.sqrt(2*(Es/No)*sin(np.pi/M)**2))
    else:
        return 2 * special.erfc(np.sqrt(2*(Es_No_Ratio)*sin(np.pi/M)**2))

def P_error_bpsk(Es_No_Ratio, Es = None, No = None):
    if Es != None and No != None:
        return 0.5*special.erfc(np.sqrt(Es_No_Ratio))
    else:
        return 0.5*special.erfc(np.sqrt(Es/No))

def calc_llr_wrong(symbol_mapping, r, s, h = np.complex64(1.0), No = 1, var = 1):
    #https://arxiv.org/pdf/1903.04656.pdf
    #I might be doing this wrong, this might supposed to be Probability(received symbol | bit of i = 1)
    null_symbols = []
    null_bits = []
    my_symbol = []
    my_bits = []
    for key,value in symbol_mapping:
        if s == key:
            my_symbol.append(key)
            my_bits.append(value)
        else:
            null_symbol.append(key)
            null_symbol.append(value)
            
    a = np.exp(-1*(np.mag(r-h*my_symbol[0]))/var) #There is only one correct symbol
    
    running_sum = 0.0
    for i in range(null_symbols):
        b = np.exp(-1*(np.mag(r-h*null_symbols[i]))/var)
        running_sum += b
    
def calc_llr_v0(symbol_mapping, r, s, h = np.complex64(1.0), No = 1, var = 1):
    #https://arxiv.org/pdf/1903.04656.pdf
    #I might be doing this wrong, this might supposed to be Probability(received symbol | bit of i = 1)
    #s is the label, r is the received symbol
    null_symbols = []
    null_bits = []
    my_symbol = []
    my_bits = []
    #print(symbol_mapping)
    for key,value in symbol_mapping.items():
        #print("for", s, key)
        if s == key:
            my_symbol.append(key)
            my_bits.append(value)
        else:
            null_symbols.append(key)
            null_bits.append(value)
    #print(my_bits)
    run_sum_1 = 0.0
    run_sum_0 = 0.0
    for bit in my_bits[0]:
        a = np.exp(-1*(np.linalg.norm(r-h*s))/var) #There is only one correct symbol
        #print(a, bit)
        if bit == '0':
            run_sum_0 += a
        else:
            run_sum_1 += a
    
    Li = np.log2(run_sum_1) - np.log2(run_sum_0)
    #If 0: Equally likely
    #print(Li)
    return Li

def has_ith_bit(i = 0, bit = '0', symbol_mapping = {}):
    sym_list = []
    for symbol, bits in symbol_mapping.items():
        if bits[i] == bit:
            sym_list.append(symbol)
    return sym_list

def calc_llr(symbol_mapping, r, h = np.complex64(1.0), var = 1):
    #https://arxiv.org/pdf/1903.04656.pdf
    #I might be doing this wrong, this might supposed to be Probability(received symbol | bit of i = 1)
    #s is the label, r is the received symbol
    Ls = []
    vals = list(symbol_mapping.values())
    for i in range(len(vals[0])):
        soft_symbols_1 = has_ith_bit(i = i, bit = '1', symbol_mapping = symbol_mapping)
        soft_symbols_0 = has_ith_bit(i = i, bit = '0', symbol_mapping = symbol_mapping)
        run_sum_1 = 0.0
        run_sum_0 = 0.0
        for local_s in soft_symbols_1:
            a = np.exp(-1*(np.linalg.norm(r-h*local_s))/var)
            run_sum_1 += a
        for local_s in soft_symbols_0:
            a = np.exp(-1*(np.linalg.norm(r-h*local_s))/var)
            run_sum_0 += a
        #print('runs1: ', run_sum_1, 'runs0: ', run_sum_0)
        Li = np.log2(run_sum_1) - np.log2(run_sum_0)
        Ls.append(Li)
    #print(Ls)
    return Ls

    
def soft_demodulate(input_symbols, symbol_mapping, h = np.complex64(1.0), var = 1):
    #Code taken from CommPy: https://github.com/veeresht/CommPy/blob/master/commpy/modulation.py
    list_syms = np.array([ k for k in symbol_mapping.keys() ])
    list_bits = np.array([ v for v in symbol_mapping.values() ])
    str_out = ''
    for i in range(len(input_symbols)):
        llrs = calc_llr(symbol_mapping, input_symbols[i], h = h, var = var)
        for llr in llrs:
            if llr <= 0.0:
                str_out += '0'
            else:
                str_out += '1'
    return str_out

In [375]:
def generateModel(in_shape = 2, bps = 2):
    model = Sequential()
    model.add(Dense(in_shape, input_shape=(in_shape, )))
    model.add(Dense(in_shape*2, input_shape=(in_shape, )))
    #leakyrelu = keras.layers.LeakyReLU(alpha=0.3)
    model.add(Activation('Relu'))
    model.add(Dense(bps, input_shape=(bps, ), 
                   kernel_regularizer=regularizers.l2(0.0001)
                    ,
                    activity_regularizer=regularizers.l1(0.0001)))
    return model

In [376]:
#Sanity Checking
print(myqam4.dict[-1-1j])
print(myqam16.dict[-1-1j])
print('calc_llr:', calc_llr(myqam4.dict, -1-1j, h = 1.0))
print('calc_llr:', calc_llr(myqam4.dict, -0.5+0.75j, h = 1.0))
print('calc_llr:', calc_llr(myqam4.dict, -0.5+0.75j, h = 1.0 + 2j))
print('calc_llr:', calc_llr(myqam16.dict, 0.75+0.75j, h = 1.0))
print('soft: ', soft_demodulate([-1-1j, -1+1j, 1+1j, 1-1j], myqam16.dict, h=1.0))
print('dict:', myqam16.dict)

00
0000
calc_llr: [-2.5457138269082145, -1.9682367243887073]
calc_llr: [1.5940402850823552, 0.8192674455538647]
calc_llr: [1.628907464555101, -0.6044061593021821]
calc_llr: [1.1757619283683902, 0.1557260165832155, -0.17371323540141548, 0.1557260165832155]
soft:  0000101010000010
dict: {(-1-1j): '0000', (-0.33333334-1j): '0001', (0.33333334-1j): '0011', (1-1j): '0010', (-1-0.33333334j): '0110', (-0.33333334-0.33333334j): '0111', (0.33333334-0.33333334j): '0101', (1-0.33333334j): '0100', (-1+0.33333334j): '1100', (-0.33333334+0.33333334j): '1101', (0.33333334+0.33333334j): '1111', (1+0.33333334j): '1110', (-1+1j): '1010', (-0.33333334+1j): '1011', (0.33333334+1j): '1001', (1+1j): '1000'}


In [377]:
#Sanity Checking
print(myPSK8.dict)
print('calc_llr:', calc_llr(myPSK8.dict, 1, h = 1.0))
print('calc_llr:', calc_llr(myPSK8.dict, -1, h = 1.0))
print('calc_llr:', calc_llr(myBPSK.dict, 1, h = 1.0))
print('calc_llr:', calc_llr(myBPSK.dict, -1, h = 1.0))
print('soft: ', soft_demodulate([1, 1, -1, -1, 1, 1], myBPSK.dict, h=1.0))
print('dict:', myBPSK.dict)

{(1+0j): '0000', (0.9238795+0.38268343j): '0001', (0.70710677+0.70710677j): '0011', (0.38268343+0.9238795j): '0010', (6.123234e-17+1j): '0110', (-0.38268343+0.9238795j): '0111', (-0.70710677+0.70710677j): '0101', (-0.9238795+0.38268343j): '0100', (-1+1.2246469e-16j): '1100', (-0.9238795-0.38268343j): '1101', (-0.70710677-0.70710677j): '1111', (-0.38268343-0.9238795j): '1110', (-1.8369701e-16-1j): '1010', (0.38268343-0.9238795j): '1011', (0.70710677-0.70710677j): '1001', (0.9238795-0.38268343j): '1000'}
calc_llr: [-0.45407425350371855, -1.6281609788657778, -0.6606638425187255, -0.19617467218996398]
calc_llr: [0.45407425350371855, 1.6281609788657778, -0.6606638425187259, -0.19617467218996398]
calc_llr: [-2.8853900817779268]
calc_llr: [2.8853900817779268]
soft:  001100
dict: {(1+0j): '0', (-1+1.2246469e-16j): '1'}


In [ ]:
#Generate randomized data now
#Separate into training and test data
%matplotlib inline
from random import choice
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
RAND_DATA_CREATE = 64**2
#RAND_DATA_CREATE = 8
REYLEIGH_FADING = np.complex64(1.0)

'''
myqam4 = QAM(NUM_SYMBOLS=4)
myqam16 = QAM(NUM_SYMBOLS=16)
myqam64 = QAM(NUM_SYMBOLS=64)
myBPSK = PSK(NUM_SYMBOLS = 2)
myPSK8 = PSK(NUM_SYMBOLS = 16)
'''
#Signals have a max amplitude of 1.
#SNR = Si - No #db
#SNR = Si - sqrt(VAR)
#VAR = (Si - SNR)**2. Assume Si = 1 (Most likely not the case)

#SNR = Si/sqrt(VAR)
#sqrt(VAR) = Si/SNR

SNRS = [-10, -2, 1, 2, 3, 5, 10, 13, 15, 20]
VARS = []
for snr in SNRS:
    snr_linear = 10**(snr / 10)
    VARS.append((1/snr)**2)

myModems = [myqam4, myqam16, myqam64, myBPSK, myPSK8]
names = ['QAM', 'QAM16', 'QAM64', 'BPSK', 'PSK8']
num_syms = [4,      16,     64,     2,        8]
num_bits = [ k for k in (np.int_(num_syms)) ]
#losses = ['categorical_crossentropy']
history_list = []
#print(num_bits)
model_list = []
data = ''.join(choice('01') for _ in range(RAND_DATA_CREATE))
Adam=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
cnt_index = 0
#print('qam64dict: ', myqam64.dict)
for modem in myModems:
    for noise_var, snr in zip(VARS, SNRS):
        print(names[cnt_index])
        myModel = generateModel(bps = modem.bit_num)
        #Process the data such that I'm sending an even number of bits through the modulator
        if (names[cnt_index] != 'BPSK'):
            length = len(data)
            num_chunks = int(np.floor_divide(length, modem.bit_num))
            proc_data = data[:length*num_chunks]
        else:
            proc_data = data
        #print('proc_data',len(proc_data),' num_chunks: ', num_chunks)

        X_pre = modem.TX(proc_data)
        X = []
        for x in X_pre:
            X.append([x.real, x.imag])
        X = np.array(X)
        y = []
        for symbol in X:
            complex_symbol = symbol[0] + 1j*symbol[1]
            y.append(calc_llr(modem.dict, complex_symbol, h=REYLEIGH_FADING, var = noise_var))
        y = np.array(y)
        #print('X\n', X, '\nY\n', y)
        print('Xshape\n', X.shape, '\nYshape\n', y.shape)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

        #print('X_train\n', X_train, '\nY_train\n', y_test)
        #print('X_test\n', X_test, '\nY_test\n', y_test)
        myModel.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'cosine_proximity', 'accuracy'])
        history=myModel.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=350, verbose=0)
        history_list.append(history)

        model_list.append(myModel)
        fig = plt.figure()

        #plt.subplot(2, 1, 1)
        plt.plot(history.history['mean_squared_error'])
        plt.plot(history.history['val_mean_squared_error'])
        plt.title('Model Accuracy %s SNR: %s' % names[cnt_index] % snr)
        plt.ylabel('MSE')
        plt.xlabel('Epoch')
        plt.legend(['MSE', 'Validation MSE'], loc = 'upper left')

        '''
        plt.subplot(2, 1, 2)
        plt.plot(history.history['val_accuracy'])
        plt.plot(history.history['accuracy'])
        plt.title('Model Accuracy %s'.format(names[cnt_index]))
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        '''
    
    plt.legend(['Validation Accuracy', 'Accuracy'], loc='upper left')
    plt.show()
    cnt_index += 1

QAM
Xshape
 (2048, 2) 
Yshape
 (2048, 2)


In [391]:
#Debug BPSK

BPSK_model = model_list[3]
print('BPSK negative:' , BPSK_model.predict(np.array([[-9.0, 0.0]])))
print('BPSK positive:' , BPSK_model.predict(np.array([[1.0, 0.0]])))

BPSK negative: [[22.902063]]
BPSK positive: [[-2.8858914]]


In [ ]:
Adam=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='cross-entropy', optimizer='Adam', metrics=['accuracy'])
training_data = np.array(X_train)
test_data = np.array(X_test)

#Run model. Make sure to regen this model to prevent overfitting
history=model.fit(training_data, y_train, validation_data=(test_data, y_test), epochs=130)
history_list.append(history)